# Web data preparation

**Author:** Florian Klaver

This notebook converts the high-resolution analysis rasters (TIFF) into optimized PNG Image Overlays for use in the Streamlit Web Application.

**Process:**
1. Reprojects rasters to Web Mercator (EPSG:3857) to ensure perfect alignment with Leaflet/Folium.
2. Applies styling (colors, transparency).
3. Downscales the Habitat Suitability map to reduce file size (Performance).
4. Saves as PNG images + JSON Bounds files.

**Outputs:**
- habitat_overlay.png (RdYlGn color map)
- scenario_wolf.png (Teal)
- scenario_human.png (Violet)
- conflict_high.png (Magenta)
- conflict_medium.png (Dark Orange)

## Setup

In [33]:
import os
import json
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling, transform_bounds
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
from pathlib import Path
from PIL import Image

In [34]:
# --- CONFIGURATION ---
try:
    script_dir = Path(__file__).parent
except NameError:
    # Fallback for Jupyter Notebooks
    script_dir = Path.cwd()

PROJECT_ROOT = script_dir.parent
OUTPUT_DIR = PROJECT_ROOT / 'output'
WEB_DATA_DIR = PROJECT_ROOT / 'web_data'

# Input Files
PATHS = {
    "wolf_scenario": OUTPUT_DIR / "scenario_best_case_wolf.tif",
    "human_scenario": OUTPUT_DIR / "scenario_best_case_human.tif",
    "conflict": OUTPUT_DIR / "conflict_zones_filtered.tif",
    "habitat": OUTPUT_DIR / "habitat_suitability_final_wlc.tif"
}

# Create output folder if it doesn't exist
WEB_DATA_DIR.mkdir(parents=True, exist_ok=True)

## Process

In [35]:
# --- FUNCTIONS ---

def convert_raster_to_png(tiff_path, output_name, color_mode="cmap", single_color=None, target_value=None, downscale_factor=1, alpha=1.0):
    """
    Converts a GeoTIFF to a PNG Image Overlay for Folium/Leaflet.
    Includes Contrast Stretching (2-98 percentile) for cmap mode
    
    Parameters:
    - tiff_path: Path to input TIFF.
    - output_name: Filename for PNG/JSON output.
    - color_mode: "cmap" (matplotlib colormap) or "single" (solid color).
    - single_color: Hex code (e.g., "#FF0000") if mode is "single".
    - target_value: If set, only pixels with this value are drawn (binary mask).
    - downscale_factor: 1 = original size, 5 = 1/5th resolution (smaller file).
    """
    print(f"Processing {tiff_path.name} -> {output_name}.png (Scale 1/{downscale_factor})...")
    
    if not tiff_path.exists():
        print(f"Error: File not found: {tiff_path}")
        return

    # 1. Reproject to Web Mercator (EPSG:3857)
    dst_crs = 'EPSG:3857'
    
    with rasterio.open(tiff_path) as src:
        # Calculate transform for Web Mercator
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        
        # Apply Downscaling to reduce image size
        dst_width = int(width / downscale_factor)
        dst_height = int(height / downscale_factor)
        
        # Recalculate transform for new dimensions
        dst_transform, dst_width, dst_height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds, 
            dst_width=dst_width, dst_height=dst_height
        )
        
        # Create destination array
        destination = np.zeros((dst_height, dst_width), dtype=np.float32)
        
        # Reproject (Warp)
        reproject(
            source=rasterio.band(src, 1),
            destination=destination,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest, # Nearest neighbor preserves sharp edges for masks
            dst_nodata=-9999
        )
        
        # Calculate exact Bounds in WGS84 (Lat/Lon) for Folium
        minx, miny, maxx, maxy = rasterio.transform.array_bounds(dst_height, dst_width, dst_transform)
        wgs84_bounds = transform_bounds(dst_crs, 'EPSG:4326', minx, miny, maxx, maxy)
        folium_bounds = [[wgs84_bounds[1], wgs84_bounds[0]], [wgs84_bounds[3], wgs84_bounds[2]]]

    # 2. Styling (Apply Colors)
    data = destination
    # Create RGBA Array (Red, Green, Blue, Alpha)
    rgba_image = np.zeros((dst_height, dst_width, 4), dtype=np.float32)
    
    if color_mode == "cmap":
        # Continuous Data (Habitat Suitability)
        mask = (data != -9999) & (~np.isnan(data))
        if mask.any():
            d_valid = data[mask]

            # SCALING (0.15 - 0.85) 
            # Ensures optimal contrast for Graubünden's data distribution.
            # RdYlBu: Low values = Red, High values = Blue.
            vmin = 0.15
            vmax = 0.85
            
            print(f"   Fixed Stretch: {vmin} (Red) to {vmax} (Blue)")
            
            # Clip data to these bounds
            d_clipped = np.clip(d_valid, vmin, vmax)
            
            # Normalize 0-1
            norm_data = (d_clipped - vmin) / (vmax - vmin)
            
            # Apply RdYlBu Colormap
            cmap = plt.get_cmap('RdYlBu')
            colors = cmap(norm_data)
            
            rgba_image[mask] = colors
            rgba_image[..., 3][mask] = alpha
            
    elif color_mode == "single":
        # Binary Mask (Scenarios / Conflicts)
        if target_value is not None:
            mask = (data == target_value)
        else:
            mask = (data == 1) # Default: Value 1 is the feature
            
        if mask.any():
            # Convert hex color to RGBA
            r, g, b, a_val = to_rgba(single_color)
            rgba_image[mask] = [r, g, b, alpha]

    # 3. Save as PNG
    # Convert 0-1 float to 0-255 uint8
    rescaled = (rgba_image * 255).astype(np.uint8)
    im = Image.fromarray(rescaled)
    
    png_path = WEB_DATA_DIR / f"{output_name}.png"
    # 'optimize=True' reduces PNG file size significantly
    im.save(png_path, format='PNG', optimize=True)
    
    # Save Bounds as JSON
    with open(WEB_DATA_DIR / f"{output_name}_bounds.json", 'w') as f:
        json.dump(folium_bounds, f)
        
    size_kb = png_path.stat().st_size / 1024
    print(f"Success: {png_path.name} saved ({size_kb:.1f} KB)")

In [36]:
# ==========================================
# MAIN EXECUTION
# ==========================================

print("\n" + "="*40)
print("WEB DATA PREPARATION (CVD SAFE)")
print("="*40)

# 1. Habitat Score: RdYlBu
convert_raster_to_png(PATHS["habitat"], "habitat_overlay", color_mode="cmap", downscale_factor=5, alpha=1.0)

# 2. Scenarios 
# Wolf 
convert_raster_to_png(PATHS["wolf_scenario"], "scenario_wolf", color_mode="single", single_color="#00FFFF", downscale_factor=2, alpha=1.0) 
# Human 
convert_raster_to_png(PATHS["human_scenario"], "scenario_human", color_mode="single", single_color="#FF02FF", downscale_factor=2, alpha=1.0) 

# 3. Conflicts 
# High 
convert_raster_to_png(PATHS["conflict"], "conflict_high", color_mode="single", single_color="#000000", target_value=2, downscale_factor=2, alpha=1.0) 
# Medium 
convert_raster_to_png(PATHS["conflict"], "conflict_medium", color_mode="single", single_color="#8B4513", target_value=1, downscale_factor=2, alpha=1.0) 

print("\nDONE.")


WEB DATA PREPARATION (CVD SAFE)
Processing habitat_suitability_final_wlc.tif -> habitat_overlay.png (Scale 1/5)...
   Fixed Stretch: 0.15 (Red) to 0.85 (Blue)
Success: habitat_overlay.png saved (6029.8 KB)
Processing scenario_best_case_wolf.tif -> scenario_wolf.png (Scale 1/2)...
Success: scenario_wolf.png saved (885.3 KB)
Processing scenario_best_case_human.tif -> scenario_human.png (Scale 1/2)...
Success: scenario_human.png saved (425.9 KB)
Processing conflict_zones_filtered.tif -> conflict_high.png (Scale 1/2)...
Success: conflict_high.png saved (349.9 KB)
Processing conflict_zones_filtered.tif -> conflict_medium.png (Scale 1/2)...
Success: conflict_medium.png saved (712.1 KB)

DONE.
